<a href="https://colab.research.google.com/github/PhiloBiblon/philobiblon-to-wikibase/blob/master/pb2wb/colab/run_postprocess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

from google.colab import auth
auth.authenticate_user()
print('Authenticated')

import os
!git clone https://github.com/PhiloBiblon/philobiblon-to-wikibase.git
os.chdir('philobiblon-to-wikibase/pb2wb')

from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload
from googleapiclient.http import MediaFileUpload

import io
from postprocess.postprocessor.generic import GenericPostprocessor



Authenticated
fatal: destination path 'philobiblon-to-wikibase' already exists and is not an empty directory.


In [ ]:
service = build('drive', 'v3')

In [ ]:
# Set variables for folder id's, bib, tables and force statements

OR_FILES_SOURCE = {'beta': '115pNT9ue480HAr996XbGRwWCLHFJK_V_',
                   'biteca': '',
                   'bitagap': ''}

POST_FILES_DESTINATION = {'beta': '1efJDT_HJoIsrRBw1bySuIke6n3xaSTyt',
                          'biteca': '',
                          'bitagap': ''}

# Set bibliography and tables to be post processed
bibliography = ['beta'] #['beta', 'bitagap', 'biteca']
tablenames = ['ms_ed', 'library', 'biography'] #['uniform_title', 'analytic', 'biography', 'library', 'copies', 'ms_ed', 'institutions', 'geography', 'bibliography', 'subject']


In [ ]:
def find_file_id(folder_id, table):
    query = f"'{folder_id}' in parents and not mimeType='application/vnd.google-apps.folder'"
    results = service.files().list(q=query,fields="nextPageToken, files(id, name)").execute()
    file_name = [item['name'] for item in results['files'] if item['name'].endswith('.qs') and table in item['name']]
    file_id = [item['id'] for item in results['files'] if item['name'].endswith('.qs') and table in item['name']]
    if len(file_id) > 0:
        return file_name[0], file_id[0]

In [ ]:
def create_folders(bib):
  directory_path = f"data/post/{bib}"
  if not os.path.exists(directory_path):
    os.makedirs(directory_path)
    print(f"Directory '{directory_path}' created successfully.")
  else:
    print(f"Directory '{directory_path}' already exists.")
  return directory_path

In [ ]:
def download_file(file_name, file_id):
    # Download the file
    if len(file_id) > 0: # Check if file_id list contains any elements
        print(f'Downloading file: {file_name} with id: {file_id}')
        request = service.files().get_media(fileId=file_id)
        fh = io.BytesIO()
        downloader = MediaIoBaseDownload(fh, request)
        done = False
        while done is False:
            status, done = downloader.next_chunk()
            print("Download %d%%." % int(status.progress() * 100))
    else:
        print(f'No file found for {file_name}') # Handle the case of an empty file_id list

    # Get the current working directory
    cwd = os.getcwd()

    # Construct the full path to the downloaded file
    file_path = os.path.join(cwd, file_name)

    # Save the downloaded file to the current working directory
    with open(file_path, 'wb') as f:
        fh.seek(0)
        f.write(fh.read())

    print(f'File downloaded to: {file_path}')

In [ ]:
def move_file(destination_id, file_name):
    file_metadata = {'name': os.path.basename(file_name)}
    media = MediaFileUpload(file_name)  # Adjust mimetype if needed
    file = service.files().create(body=file_metadata, media_body=media, fields='id').execute()
    file_id = file['id']

    # Move the uploaded file to the desired folder
    file = service.files().get(fileId=file_id, fields='parents').execute()
    previous_parents = ",".join(file.get('parents'))
    file = service.files().update(fileId=file_id,
                                  addParents=destination_id,
                                  removeParents=previous_parents,
                                  fields='id, parents').execute()

    print(f'File {file_name} copied successfully! File ID: {file_id}')

In [ ]:
force_new_statements = True

for bib in bibliography:
  print(f'Processing for {bib} bibliography')
  directory_path = create_folders(bib)
  for item in tablenames:
    print(f'Starting processing {item} table')
    folder_id = OR_FILES_SOURCE[bib]
    post_folder_id = POST_FILES_DESTINATION[bib]

    try:
      file_name, file_id = find_file_id(folder_id, item)
    except:
      print(f'No file found for {item}')
      continue

    download_file(file_name, file_id)
    GenericPostprocessor().postprocess(file_name, directory_path, force_new_statements)
    print(f'postprocess complete for {item}')
    print('................................')

    # Move processed file to Drive
    #move_file(post_folder_id, directory_path + '/' + file_name, file_id)
    move_file(post_folder_id, directory_path + '/' + file_name)

  print('Post processing complete')



Processing for beta bibliography
Directory 'data/post/beta' already exists.
Starting processing ms_ed table
Download 100%.
File downloaded to: /content/philobiblon-to-wikibase/pb2wb/beta_ms_ed.qs
True
postprocess complete for ms_ed
................................
File data/post/beta/beta_ms_ed.qs copied successfully! File ID: 1wtajIZNhfo4yPCWHmYP_-gzTRfhWjYvx
Starting processing library table
Download 100%.
File downloaded to: /content/philobiblon-to-wikibase/pb2wb/beta_library.qs
True
postprocess complete for library
................................
File data/post/beta/beta_library.qs copied successfully! File ID: 1HvK_v-phEfC_6_gEP9zItYoRW2ZA7hvu
Starting processing biography table
Download 100%.
File downloaded to: /content/philobiblon-to-wikibase/pb2wb/beta_biography.qs
True
postprocess complete for biography
................................
File data/post/beta/beta_biography.qs copied successfully! File ID: 1lO9j4A4pNCl18A8IKXy2Fv8r3mEkxrUs
Post processing complete
